<a href="https://colab.research.google.com/github/WebiksInc/data-explorer/blob/add-stop-word/load_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import plotly.express as px
import warnings
warnings.filterwarnings('ignore')

path_corpus = r'C:\corpus'

In [ ]:
from flair.data import Sentence
from flair.models import SequenceTagger

RuntimeError: Failed to import transformers.trainer_utils because of the following error (look up to see its traceback):
Could not find the DLL(s) 'msvcp140_1.dll'. TensorFlow requires that these DLLs be installed in a directory that is named in your %PATH% environment variable. You may install these DLLs by downloading "Microsoft C++ Redistributable for Visual Studio 2015, 2017 and 2019" for your platform from this URL: https://support.microsoft.com/help/2977003/the-latest-supported-visual-c-downloads

In [ ]:
#####################
####OpenSubtitles####
#####################

# dir_corp = r'C:\corpus\OpenSubtitles\he.txt'
# row_counter = 0
# dictionary_list = []
# print_count = 5000000
# with open(dir_corp,encoding = 'utf8') as myfile:
#     df = pd.DataFrame([], columns=['line'])
#     for line in myfile:
#         row_counter = row_counter + 1
#         # if row_counter >= 50000:
#         #     break
#         if row_counter % print_count == 0:
#             print(row_counter/print_count)
#         dictionary_list.append(line)
#
#
# df = pd.DataFrame({'line':dictionary_list})
#
# df = df.replace({'\n': ''},regex=True)
#
# # df.to_pickle('C:\corpus\CC-100\OpenSubtitles pickle')
# # df = pd.read_pickle('C:\corpus\OpenSubtitles\OpenSubtitles pickle')

1.0
2.0
3.0
4.0
5.0
6.0
7.0
8.0
9.0
10.0
11.0
12.0
13.0
14.0
15.0
16.0
17.0


In [ ]:
# ###################
# #### wikipadie ####
# ###################
# import xml.etree.ElementTree as etree
# import codecs
# import csv
# import time
# import os
#
#
# pathWikiXML = r'C:\corpus\wikimedia\hewiki-latest-pages-articles.xml'
# pathWikiCSV = r"C:\corpus\wikimedia\Output_temp.csv"
#
# def strip_tag_name(t):
#     return t.split("}")[1] if "}" in t else t
#
# cnt =0
# with codecs.open(pathWikiCSV, "w", "utf-8") as f:
#     cw = csv.writer(f)
#     cw.writerow(['title', 'timestamp','text','page'])
#
#     for event, elem in etree.iterparse(pathWikiXML, events=('start', 'end')):
#         tname = strip_tag_name(elem.tag)
#         if event == 'start':
#             if tname == 'page':
#                 cnt = cnt+1
#                 title = None
#                 timestamp = None
#                 username = None
#                 ip = None
#                 text = None
#
#             elif tname == 'title':
#                 title = elem.text
#             elif tname == 'timestamp':
#                 timestamp = elem.text
#             elif tname == 'username':
#                 username = elem.text
#             elif tname == 'ip':
#                 ip = elem.text
#             elif tname == 'text':
#                 text = elem.text
#             elif tname == 'page':
#                 page = elem.text
#                 cw.writerow([title, timestamp,  text, page])
#             if cnt > 100:
#                 break
#         elem.clear()
#
# df = pd.read_csv( r"C:\corpus\wikimedia\Output_temp.csv")
# df.sample(2)

In [ ]:
####################
#### ben yehuda ####
####################
# TODO: compare to Project Gutenberg. The problem is that is 40 GB


df = pd.read_csv(path_corpus + r'\ben yehuda\pseudocatalogue.csv')
lst = df['path']
path_load = path_corpus + r'\ben yehuda\txt'

def read_first_line(file_path):
    with open(file_path, 'r',encoding = 'utf8') as file:
        return file.read().replace('\n', ' ').replace('\t', ' ')


dictionary_list = []
for i in lst:
    i = i.replace('/','\\')
    path = path_load + i + '.txt'
    dictionary_list.append(read_first_line(path))

df = pd.DataFrame({'line':dictionary_list})
df.to_csv(path_corpus + r'\ben yehuda\df.csv')

In [ ]:
# https://cognitivedemons.wordpress.com/2017/07/10/downloading-all-english-books-from-gutenberg-org-with-python/

import requests, bs4, os, errno, zipfile, glob
from urllib.request import urlretrieve


def main():

    if not os.path.exists('books/'):
        try:
            os.makedirs('books/')
        except OSError as e:
            if e.errno != errno.EEXIST:
                raise

    # STEP 1. BUILD A LIST OF URLS

    urls_to_books = []

    if not os.path.exists('urls_to_books.txt'):

        page_w_books_url = 'http://www.gutenberg.org/robot/harvest?filetypes[]=txt&langs[]=en'

        while 1 == 1:

            is_last_page = False

            print('Reading page: ' + page_w_books_url)

            page_w_books = requests.get(page_w_books_url, timeout=20.0)

            if page_w_books:
                page_w_books = bs4.BeautifulSoup(page_w_books.text, "lxml")
                urls = [el.get('href') for el in page_w_books.select('body > p > a[href^="http://aleph.gutenberg.org/"]')]
                url_to_next_page = page_w_books.find_all('a', string='Next Page')

                if len(urls) > 0:
                    urls_to_books.append(urls)

                    if url_to_next_page[0]:
                        page_w_books_url = "http://www.gutenberg.org/robot/" + url_to_next_page[0].get('href')
                else:
                    is_last_page = True

            if is_last_page:
                break

        urls_to_books = [item for sublist in urls_to_books for item in sublist]

        # Backing up the list of URLs
        with open('urls_to_books.txt', 'w') as output:
            for u in urls_to_books:
                output.write('%s\n' % u)

    # STEP 2. DOWNLOAD BOOKS

    # If, at some point, Step 2 is interrupted due to unforeseen
    # circumstances (power outage, lost of internet connection), replace the number
    # (value of the variable url_num) below with the one you will find in the logfile.log
    # Example
    #       logfile.log : Unzipping file #99 books/10020.zip
    #       the number  : 99
    url_num = 0

    if os.path.exists('urls_to_books.txt') and len(urls_to_books) == 0:
        with open('urls_to_books.txt', 'r') as f:
            urls_to_books = f.read().splitlines()

    for url in urls_to_books[url_num:]:

        dst = 'books/' + url.split('/')[-1].split('.')[0].split('-')[0]

        with open('logfile.log', 'w') as f:
            f.write('Unzipping file #' + str(url_num) + ' ' + dst + '.zip' + '\n')

        if len(glob.glob(dst + '*')) == 0:
            urlretrieve(url, dst + '.zip')

            with zipfile.ZipFile(dst + '.zip', "r") as zip_ref:
                try:
                    zip_ref.extractall("books/")
                    print(str(url_num) + ' ' + dst + '.zip ' + 'unzipped successfully!')
                except NotImplementedError:
                    print(str(url_num) + ' Cannot unzip file:', dst)

            os.remove(dst + '.zip')

        url_num += 1


if __name__ == '__main__':
    """
    The main function is called when gutenberg.py is run from the command line
    """

    main()

0 books/comed10.zip unzipped successfully!
1 books/12370.zip unzipped successfully!
3 books/12372.zip unzipped successfully!
5 books/12373.zip unzipped successfully!
7 books/12374.zip unzipped successfully!
9 books/12375.zip unzipped successfully!
11 books/12376.zip unzipped successfully!
13 books/12377.zip unzipped successfully!
14 books/12378.zip unzipped successfully!
16 books/12380.zip unzipped successfully!
18 books/12381.zip unzipped successfully!
19 books/12383.zip unzipped successfully!
21 books/12384.zip unzipped successfully!
23 books/12385.zip unzipped successfully!
25 books/12386.zip unzipped successfully!
26 books/10001.zip unzipped successfully!
27 books/10002.zip unzipped successfully!
29 books/10003.zip unzipped successfully!
30 books/10004.zip unzipped successfully!
32 books/10005.zip unzipped successfully!
34 books/10006.zip unzipped successfully!
36 books/10008.zip unzipped successfully!
38 books/10009.zip unzipped successfully!
39 books/10010.zip unzipped successful

ContentTooShortError: <urlopen error retrieval incomplete: got only 65547 out of 166604 bytes>

In [ ]:
import os
import random
data = []
path = r'C:\Users\user\AppData\Roaming\JetBrains\DataSpell2021.3\projects\workspace\books'
files = [f for f in os.listdir(path) if file.endswith(".txt")]
files = random.sample(files,200)
for f in files:
    with open(os.path.join(path,f), "r",encoding = 'ISO-8859-1') as myfile:
        data.append(myfile.read())

df = pd.DataFrame(data)
df.head()
df.to_csv(r'C:\corpus\gutenberg\df.csv')

In [ ]:
# ##################
# #### think il ####
# ##################
# from bs4 import BeautifulSoup
# import requests
# import time
#
# path_corpus = path_corpus + '\\think il'
#
# url="https://thinkil.co.il/texts-sitemap.xml"
# res=requests.get(url, verify=False)
# soup = BeautifulSoup(res.content, ['lxml', 'xml'])
# soup_list = soup.findChildren('loc')
# urls_list = []
# for element in soup_list:
#     net = element.get_text()
#     if net.endswith(('jpg','png')) == False:
#         urls_list.append(net)
# #remove the first link
# dat = pd.DataFrame(columns=['url','title','auther','text'])
# dat.url = urls_list[1:]
#
# for i in range(len(dat)):
#     time.sleep(1)
#     res = requests.get(dat.loc[i,'url'])
#     res.content
#     soup = BeautifulSoup(res.content, 'html.parser')
#     title_author = soup.findChildren('hgroup')[0].get_text().split('מאת:')
#     title = title_author[0]
#     author = title_author[1]
#     text = soup.findChildren('article')[0].get_text()
#     dat.loc[i,1:] = [title,author,text]
#
# dat = dat.replace(['\n'],' ',regex=True)
# dat.to_csv(path_corpus + '\\df think IL.tsv', sep='\t')

In [ ]:
def stanza_tokenizer(x: str)-> str:
    lst = []
    doc = nlp(x)
    for sentence in doc.sentences:
        for word in sentence.words:
            lst.append(word.text)
    return ' '.join(lst)

from tqdm import tqdm
import stanza
nlp = stanza.Pipeline(lang='he', processors='tokenize')
tqdm.pandas()
dat['text'] = dat['text'].progress_apply(lambda x: stanza_tokenizer(x))

dat.to_csv(path_corpus + '\\df think IL token.tsv', sep='\t')

In [ ]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\user\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping tokenizers\punkt.zip.


True

In [ ]:
from nltk.tokenize import word_tokenize
dfcopy = df[0].apply(lambda text: ' '.join(word_tokenize(text)))

In [ ]:
dfcopy.to_csv(r'C:\corpus\gutenberg\df toknazie.csv')

In [ ]:
## subset wikipedia with random
import wikipedia
wikipedia.set_rate_limiting(True)
n_artc = 1000
for lng in ['en','he']:
    wikipedia.set_lang(lng)
    lst_artc = wikipedia.random(n_artc)
    artc_cntnt = []
    for artc in lst_artc:
        try:
            temp = wikipedia.page(artc)
            artc_cntnt.append(temp.content)
        except:
            artc_cntnt.append([])

    temp = pd.DataFrame({'title':lst_artc,'content':artc_cntnt})
    temp.to_csv(path_corpus + r'\subset wiki\ ' + lng +  ' df.csv',sep='\t')

In [ ]:
path_mila = r'C:\corpus\mila'

def save_mila(url:str,source:str):
    """"
    General function to Save 10 values from a MILA
    """
    import requests
    from bs4 import BeautifulSoup
    response = requests.get(url)
    soup = BeautifulSoup(response.content,"html.parser")
    url_base = 'https://yeda.cs.technion.ac.il:8443'
    data = []
    i = 0
    for link in soup.find_all('a'):
        url_spe = link.get('href')
        if url_spe.endswith(('.txt','.out')):
            url_update = url_base + url_spe
            r  = requests.get(url_update)
            r.encoding = r.apparent_encoding
            data.append(r.text)
#             if i == 10:
#                 break
#             i+=1
    temp = pd.DataFrame({'text':data})
    temp.to_csv(path_mila + '\\' + source + ' df.csv',sep='\t')

In [ ]:
save_mila('https://yeda.cs.technion.ac.il:8443/corpus/software/corpora/haaretz/txt/apr15-23-90/','haaretz')
save_mila('https://yeda.cs.technion.ac.il:8443/corpus/software/corpora/a7corpus/archive/txt/articles/1000-1099','Arutz 7')
save_mila('https://yeda.cs.technion.ac.il:8443/corpus/software/corpora/themarker/cp1255txt/20020512/','TheMarker')
save_mila('https://yeda.cs.technion.ac.il:8443/corpus/software/corpora/knesset/txt/utf8/16/','HaKnesset')
save_mila('https://yeda.cs.technion.ac.il:8443/corpus/software/corpora/doctors/txt/doctors/','Doctors')
save_mila('https://yeda.cs.technion.ac.il:8443/corpus/software/corpora/infomed/txt/06/','Infomed')
save_mila('https://yeda.cs.technion.ac.il:8443/corpus/software/corpora/natureHilling/txt/natureHilling/','Nature of Healing')
save_mila('https://yeda.cs.technion.ac.il:8443/corpus/software/corpora/2b/txt/articles/','To Be Healthy')
save_mila('https://yeda.cs.technion.ac.il:8443/corpus/software/corpora/tapuz/txt/text2/','Tapuz People Forum')
save_mila('https://yeda.cs.technion.ac.il:8443/corpus/software/corpora/spokenHebrew/txt/','Spoken Israeli Hebrew')
save_mila('https://yeda.cs.technion.ac.il:8443/corpus/software/corpora/shaar/txt/shaar/','Hebrew Dotted Text')
save_mila('https://yeda.cs.technion.ac.il:8443/corpus/software/corpora/foodWalla/txt/','Walla Food Corpus')
save_mila('https://yeda.cs.technion.ac.il:8443/corpus/software/corpora/foodpage/txt/','Foodpage Corpus')
save_mila('https://yeda.cs.technion.ac.il:8443/corpus/software/corpora/sportWalla/txt/','Walla Sport Corpus')
save_mila('https://yeda.cs.technion.ac.il:8443/corpus/software/corpora/sport5/txt/','Sport5 Corpus')
save_mila('https://yeda.cs.technion.ac.il:8443/corpus/software/corpora/learningMan/txt/','Learning Man')

In [ ]:
# ParaShoot
import requests
from bs4 import BeautifulSoup
response = requests.get(r'https://raw.githubusercontent.com/omrikeren/ParaShoot/main/data/train.json')
temp = response.json()
pd.DataFrame(temp['data']).to_csv(r'C:\corpus\ParaShoot\df.csv',sep='\t')

In [ ]:
# NER
# https://www.cs.bgu.ac.il/~elhadad/nlpproj/naama/
import requests
import pandas as pd
r  = requests.get('https://www.cs.bgu.ac.il/~elhadad/nlpproj/naama/tagged_corpus.txt')
r.encoding = r.apparent_encoding
temp = r.text.split("\n")
def split_txt(x:str):
    if ' ' in x:
        temp = x.split(' ')
        return (temp[0], temp[1])
temp = [split_txt(x) for x in temp]
path_corpus = r'C:\corpus'
temp = pd.DataFrame(temp)
temp.columns = ['line','entity']
temp.to_csv(path_corpus + r'\NER Naama Ben-Mordecai\df.csv',sep='\t')